<h1 align=center><font size = 5>Regression Models with Keras</font></h1>

## Introduction

Keras is a high-level API for building deep learning models. It has gained favor for its ease of use and syntactic simplicity facilitating fast development. Building a very complex deep learning network can be achieved with Keras with only few lines of code.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Download and Clean Dataset</a>  
2. <a href="#item32">Import Keras</a>  
3. <a href="#item33">Build a Neural Network</a>  
4. <a href="#item34">Train and Test the Network</a>  

</font>
</div>

<a id="item31"></a>

## Download and Clean Dataset

<strong>The dataset contains the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

Let's start by importing the <em>pandas</em> and the Numpy libraries.

In [2]:
import pandas as pd
import numpy as np

/home/jupyterlab/conda/envs/python/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


Let's download the data and read it into a <em>pandas</em> dataframe.

In [3]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

#### Let's check how many data points we have.

In [4]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.

In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

#### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>

Let's do a quick sanity check of the predictors and the target dataframes.

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let us split the data into training and test using sklearn

In [11]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train, y_test = train_test_split(predictors_norm, target)

X_train.shape,X_test.shape,y_train.shape,y_test.shape

((772, 8), (258, 8), (772,), (258,))

Let's save the number of predictors to *n_cols* since we will need this number when building our network.

In [12]:
n_cols = X_train.shape[1] # number of predictors

<a id="item1"></a>

## Import Keras

Keras normally runs on top of a low-level library such as TensorFlow. This means that to be able to use the Keras library, we will have to install TensorFlow first and when we import the Keras library, it will be explicitly displayed what backend was used to install the Keras library.

#### Let's go ahead and import the Keras library

In [13]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [14]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>

## Build a Neural Network

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

In [15]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has two hidden layers, each of 50 hidden units.

<a id="item4"></a>

## Train and Test the Network

Let's call the function now to create our model.

In [16]:
# build the model
model = regression_model()

Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 100 epochs.

In [17]:
# fit the model
model.fit(X_train, y_train, validation_split=0.3, epochs=100, verbose=2)

Train on 540 samples, validate on 232 samples
Epoch 1/100
 - 1s - loss: 1534.1363 - val_loss: 1396.0201
Epoch 2/100
 - 0s - loss: 1445.9011 - val_loss: 1298.2356
Epoch 3/100
 - 0s - loss: 1326.0363 - val_loss: 1155.7860
Epoch 4/100
 - 0s - loss: 1151.2015 - val_loss: 966.9143
Epoch 5/100
 - 0s - loss: 931.5605 - val_loss: 743.1083
Epoch 6/100
 - 0s - loss: 691.4135 - val_loss: 523.7628
Epoch 7/100
 - 0s - loss: 479.8930 - val_loss: 347.8141
Epoch 8/100
 - 0s - loss: 333.6994 - val_loss: 247.4143
Epoch 9/100
 - 0s - loss: 258.3067 - val_loss: 202.9052
Epoch 10/100
 - 0s - loss: 230.0463 - val_loss: 183.2262
Epoch 11/100
 - 0s - loss: 216.7367 - val_loss: 173.8624
Epoch 12/100
 - 0s - loss: 207.5446 - val_loss: 166.3667
Epoch 13/100
 - 0s - loss: 200.4555 - val_loss: 161.0720
Epoch 14/100
 - 0s - loss: 193.4473 - val_loss: 155.8905
Epoch 15/100
 - 0s - loss: 187.9181 - val_loss: 151.9405
Epoch 16/100
 - 0s - loss: 183.5067 - val_loss: 149.1329
Epoch 17/100
 - 0s - loss: 179.1779 - val_lo

Let us evaluate our trained model on the test dataset using *model.evaluate*

In [19]:
scores = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: ', scores)

Accuracy:  65.42166468154552


<hr>